In [47]:
import time
import pandas as pd
import json
import requests
import aiohttp
import asyncio
import numpy as np
import pendulum

from pathlib import Path

## Note:
See `240220_sl_surf_spots.ipynb` for spot getter

In [2]:
response = requests.get("https://services.surfline.com/taxonomy?type=taxonomy&id=58f7ed51dadb30820bb3879c&maxDepth=0")

* You will not get Surfline forecast data without a valid Surfline premium login. Add your credentials to `.env.development`:
  ```
  SURFLINE_EMAIL=xxx
  SURFLINE_PASSWORD=yyy
  ```

##### Requests

`https://services.surfline.com/kbyg/spots/forecasts/{type}?{params}`


Type|Data
----|----
rating|array of human-readable and numeric (0-6) ratings
wave|array of min/max sizes & optimal scores
wind|array of wind directions/speeds & optimal scores
tides|array of types & heights
weather|array of sunrise/set times, array of temperatures/weather conditions

Param|Values|Effect
-----|------|------
spotId|string|Surfline spot id that you want data for. A typical Surfline URL is `https://www.surfline.com/surf-report/venice-breakwater/590927576a2e4300134fbed8` where `590927576a2e4300134fbed8` is the `spotId`
days|integer|Number of forecast days to get (Max 6 w/o access token, Max 17 w/ premium token)
intervalHours|integer|Minimum of 1 (hour)
maxHeights|boolean|`true` seems to remove min & optimal values from the wave data output
sds|boolean|If true, use the new LOTUS forecast engine
accesstoken|string|Auth token to get premium data access (optional)

Anywhere there is an `optimalScore` the value can be interpreted as follows:

Value|Meaning
-----|-------
0|Suboptimal
1|Good
2|Optimal


In [3]:
types = ["rating", "wave", "wind", "tides", "weather"]
params = ["spotId", "days", "intervalHours", "maxHeights", "sds", "accesstoken"]
base = "https://services.surfline.com/kbyg/spots/forecasts"

In [4]:
datapath = Path('./data')


In [5]:
df = pd.read_csv(datapath/'spot_list.csv')


In [6]:
df.head()

,Unnamed: 0,ids,names,lon,lat,urls
0,0,584204204e65fad6a7709b5d,Dauphin Island,-88.11700,30.229000,https://www.surfline.com/surf-report/dauphin-i...
1,1,584204204e65fad6a7709b61,Spuds,-87.54900,30.273000,https://www.surfline.com/surf-report/spuds/584...
2,2,584204204e65fad6a7709b62,Alabama Point,-87.56200,30.270000,https://www.surfline.com/surf-report/alabama-p...
3,3,584204204e65fad6a7709b60,West Pass,-87.73700,30.239000,https://www.surfline.com/surf-report/west-pass...
4,4,65948156c329a78a0914a15e,Morgantown Beach,-87.91913,30.230299,https://www.surfline.com/surf-report/morgantow...


Get the spot `id` for 1st Street Jetty in Va Beach

In [8]:
jetty_id = df[df['names'].str.contains('1st Street Jetty', case=False, na=False)]['ids'].values[0]
jetty_id

'584204214e65fad6a7709ce7'

In [9]:
ex_params = {params[0]: jetty_id}
ex_params

{'spotId': '584204214e65fad6a7709ce7'}

Surfline seems to change their spot IDs periodically. Check a spot on the website and pass the objectId from the url as a param to debug if this is the case. If they've changed you'll need to run the notebook `240220_sl_surf_spots.ipynb` as mentioned above to refresh the spots dataset

In [10]:
debug_params = {params[0]: "584204214e65fad6a7709ce7"}

In [11]:
res = requests.get(f"{base}/{types[0]}", params=ex_params)
res.status_code

200

In [34]:
rating_json = res.json()

In [32]:
four_day_json = res.json()
if 'data' in four_day_json and 'rating' in four_day_json['data']:
    four_day_json['data']['rating'] = four_day_json['data']['rating'][:24]

In [35]:
def cull_extra_days(full_json):
    if 'data' in full_json and 'rating' in full_json['data']:
        full_json['data']['rating'] = full_json['data']['rating'][:24]

Drop extra days of forecast

In [36]:
cull_extra_days(four_day_json)

In [41]:
new_dict

{'spot_id': 'test',
 'spot_name': 'test_2',
 'date': DateTime(2024, 5, 27, 18, 2, 33, 608271, tzinfo=Timezone('UTC')),
 'forecast': {'associated': {'location': {'lon': -88.117, 'lat': 30.229},
   'runInitializationTimestamp': 1716789600},
  'data': {'rating': [{'timestamp': 1716786000,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716789600,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716793200,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716796800,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716800400,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716804000,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716807600,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716811200,
     'utcOffset': -5,
     'rating

In [37]:
len(four_day_json['data']['rating'])

24

Convert a unix timestamp -> utc

In [13]:
pendulum.from_timestamp(rating_json['data']['rating'][0]['timestamp'], 'UTC')

DateTime(2024, 5, 27, 4, 0, 0, tzinfo=Timezone('UTC'))

In [14]:
pendulum.from_timestamp(rating_json['data']['rating'][int(72 / 3)-1]['timestamp'], 'UTC')

DateTime(2024, 5, 28, 3, 0, 0, tzinfo=Timezone('UTC'))

The `utcOffset` field seems to be aware that I'm working in EST currently. Either that or it's the time coding for the spot itself.

Let's check a west coast spot to confirm how this is handled

In [20]:
df

,Unnamed: 0,ids,names,lon,lat,urls
0,0,584204204e65fad6a7709b5d,Dauphin Island,-88.117000,30.229000,https://www.surfline.com/surf-report/dauphin-i...
1,1,584204204e65fad6a7709b61,Spuds,-87.549000,30.273000,https://www.surfline.com/surf-report/spuds/584...
2,2,584204204e65fad6a7709b62,Alabama Point,-87.562000,30.270000,https://www.surfline.com/surf-report/alabama-p...
3,3,584204204e65fad6a7709b60,West Pass,-87.737000,30.239000,https://www.surfline.com/surf-report/west-pass...
4,4,65948156c329a78a0914a15e,Morgantown Beach,-87.919130,30.230299,https://www.surfline.com/surf-report/morgantow...
...,...,...,...,...,...,...
1298,1298,640a2d14451905376297f483,Rutherford Beach,-93.124300,29.758500,https://www.surfline.com/surf-report/rutherfor...
1299,1299,5842041f4e65fad6a7708a1a,Assateague,-75.177040,38.148058,https://www.surfline.com/surf-report/assateagu...
1300,1300,5842041f4e65fad6a770886d,Ocean City Boardwalk,-75.081170,38.338461,https://www.surfline.com/surf-report/ocean-cit...
1301,1301,5842041f4e65fad6a7708a1b,North End to Ocean City Inlet,-75.080177,38.338890,https://www.surfline.com/surf-report/north-end...


In [15]:
la_jolla_id = df[df['names'].str.contains("La Jolla", case=False, na=False)]['ids'].values[0]
la_jolla_dict = {params[0]: la_jolla_id}

In [21]:
la_jolla_dict

{'spotId': '5842041f4e65fad6a77088cc'}

In [28]:
pendulum.now("utc")

DateTime(2024, 5, 27, 17, 47, 27, 97332, tzinfo=Timezone('UTC'))

In [40]:
new_dict = {"spot_id": "test", "spot_name": "test_2", "date": pendulum.now("utc"), "forecast": four_day_json}

In [52]:
spot_ratings = []
for spot_id, spot_name in df[['ids', 'names']][:3].values:
    res = requests.get(f"{base}/rating", params={'spotId': spot_id})
    data = res.json()
    cull_extra_days(data)
    current_date = pendulum.now("utc")
    utc_date = current_date.strftime("%Y-%m-%d")
    expanded_data = {"spot_id": spot_id, "spot_name": spot_name, "utc_date": utc_date, "forecast": data}
    spot_ratings.append(expanded_data)
    time.sleep()

In [53]:
spot_ratings

[{'spot_id': '584204204e65fad6a7709b5d',
  'spot_name': 'Dauphin Island',
  'utc_date': '2024-05-27',
  'forecast': {'associated': {'location': {'lon': -88.117, 'lat': 30.229},
    'runInitializationTimestamp': 1716789600},
   'data': {'rating': [{'timestamp': 1716786000,
      'utcOffset': -5,
      'rating': {'key': 'POOR', 'value': 1}},
     {'timestamp': 1716789600,
      'utcOffset': -5,
      'rating': {'key': 'POOR', 'value': 1}},
     {'timestamp': 1716793200,
      'utcOffset': -5,
      'rating': {'key': 'POOR', 'value': 1}},
     {'timestamp': 1716796800,
      'utcOffset': -5,
      'rating': {'key': 'POOR', 'value': 1}},
     {'timestamp': 1716800400,
      'utcOffset': -5,
      'rating': {'key': 'POOR', 'value': 1}},
     {'timestamp': 1716804000,
      'utcOffset': -5,
      'rating': {'key': 'POOR', 'value': 1}},
     {'timestamp': 1716807600,
      'utcOffset': -5,
      'rating': {'key': 'POOR', 'value': 1}},
     {'timestamp': 1716811200,
      'utcOffset': -5,
    

In [26]:
spot_ratings

[{'associated': {'location': {'lon': -88.117, 'lat': 30.229},
   'runInitializationTimestamp': 1716789600},
  'data': {'rating': [{'timestamp': 1716786000,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716789600,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716793200,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716796800,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716800400,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716804000,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716807600,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716811200,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
    {'timestamp': 1716814800,
     'utcOffset': -5,
     'rating': {'key': 'POOR', 'value': 1}},
 

In [17]:
rating_json = res.json()

In [18]:
pendulum.from_timestamp(rating_json['data']['rating'][0]['timestamp'], 'UTC')

DateTime(2024, 5, 27, 7, 0, 0, tzinfo=Timezone('UTC'))

Alright, so it looks like each spot's forecast starts at 12am *local time*, with the timestamp for that time in unix. To figure out the flat `UTC` time for each spot you can just apply the `utcOffset` that is included in response. 

In [19]:
rating_json

{'associated': {'location': {'lon': -117.257, 'lat': 32.863},
  'runInitializationTimestamp': 1716789600},
 'data': {'rating': [{'timestamp': 1716793200,
    'utcOffset': -7,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1716796800,
    'utcOffset': -7,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1716800400,
    'utcOffset': -7,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1716804000,
    'utcOffset': -7,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1716807600,
    'utcOffset': -7,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1716811200,
    'utcOffset': -7,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1716814800,
    'utcOffset': -7,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1716818400,
    'utcOffset': -7,
    'rating': {'key': 'POOR_TO_FAIR', 'value': 2}},
   {'timestamp': 1716822000,
    'utcOffset': -7,
    'rati

***

### Ratings